<a href="https://colab.research.google.com/github/marianeneiva/handlingMissingData/blob/main/predictMissingLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PREDICTING MISSING VALUES WITH LINEAR REGRESSION

Previously *(on < insert your tv show here.. >... kidding)* I posted the article with [four ways to handle missing data with pandas](https://maribneiva.medium.com/4-ways-to-handle-missing-data-with-pandas-420a103a2105).

At the end I gave a spoiler *(am I watching too much Netflix?)* about the next episode of my Medium and here it is! 

Today we will learn how to predict missing values using machine learning (more specifically: linear regression


#Downloading data

The first step is to download the data from https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home. I downloaded from Kaggle API but you can skip this step and download it manually.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/portfolio_dataScience"

#changing the working directory
%cd /content/gdrive/My Drive/portfolio_dataScience
#Check the present working directory using pwd command

!kaggle datasets download -d dansbecker/melbourne-housing-snapshot
!ls

#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/portfolio_dataScience
  0% 0.00/451k [00:00<?, ?B/s]
100% 451k/451k [00:00<00:00, 30.3MB/s]
kaggle.json  melb_data.csv  melbourne-housing-snapshot.zip
Archive:  melbourne-housing-snapshot.zip
replace melb_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


#Importing data

The previous step download the data to my google drive. Now it is time to import it!

In [ ]:
import pandas as pd 

data = pd.read_csv('melb_data.csv')

#showing the data
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


#Transforming the data

Although some machine learning methods are able to deal with categorial data (strings, dates, among others), this is not the case of linear regression. 

Therefore, one import step is to transform all categorical attributes to numbers.

The code below uses pandas to perform the required action

In [ ]:
data.Address = pd.Categorical(data.Address).codes
data.Suburb = pd.Categorical(data.Suburb).codes
data.Type = pd.Categorical(data.Type).codes
data.Method = pd.Categorical(data.Method).codes
data.SellerG = pd.Categorical(data.SellerG).codes
data.Date = pd.Categorical(data.Date).codes
data.CouncilArea = pd.Categorical(data.CouncilArea).codes
data.Regionname = pd.Categorical(data.Regionname).codes

#Separate columns

In this example, I will use only columns that does not contain null values. To do this automatically, we can use the isnull() function aligned with mean() (sum() would work as well).


In [ ]:
dataNull = data[data.columns[data.isnull().mean() > 0]]
dataNotNull = data[data.columns[data.isnull().mean() == 0]]

print(dataNull)
print(dataNotNull)

       Car  BuildingArea  YearBuilt
0      1.0           NaN        NaN
1      0.0          79.0     1900.0
2      0.0         150.0     1900.0
3      1.0           NaN        NaN
4      2.0         142.0     2014.0
...    ...           ...        ...
13575  2.0           NaN     1981.0
13576  2.0         133.0     1995.0
13577  4.0           NaN     1997.0
13578  5.0         157.0     1920.0
13579  1.0         112.0     1920.0

[13580 rows x 3 columns]
       Suburb  Address  Rooms  ...  Longtitude  Regionname  Propertycount
0           0    12794      2  ...   144.99840           2         4019.0
1           0     5943      2  ...   144.99340           2         4019.0
2           0     9814      3  ...   144.99440           2         4019.0
3           0     9004      3  ...   144.99690           2         4019.0
4           0    10589      4  ...   144.99410           2         4019.0
...       ...      ...    ...  ...         ...         ...            ...
13575     302     1991  

#Training and test sets

Yes, we could use k-fold or any other validation step to improve the confidence in the results. Also, we could use correlation analysis to chose the columns that are more related to the one we chose to predict.

Yes, does additions would be great and I suggest you to keep them in mind for a bigger project. 

However, for the sake of simplicity, I will keep it basic for this post.

Furthermore, in this example, we will predict BuildingArea column and the code is how I separate training and test sets.

In [ ]:
import numpy as np

y_test = dataNull[dataNull['BuildingArea'].isnull()]['BuildingArea']
y_train = dataNull[~dataNull['BuildingArea'].isnull()]['BuildingArea']
print(y_test)
print(y_train)

x_training = dataNotNull[~data['BuildingArea'].isnull()]
x_test = dataNotNull[data['BuildingArea'].isnull()]

print(x_training)
print(x_test)

0       NaN
3       NaN
5       NaN
8       NaN
10      NaN
         ..
13569   NaN
13570   NaN
13574   NaN
13575   NaN
13577   NaN
Name: BuildingArea, Length: 6450, dtype: float64
1         79.0
2        150.0
4        142.0
6        210.0
7        107.0
         ...  
13572     79.0
13573    172.0
13576    133.0
13578    157.0
13579    112.0
Name: BuildingArea, Length: 7130, dtype: float64
       Suburb  Address  Rooms  ...  Longtitude  Regionname  Propertycount
1           0     5943      2  ...   144.99340           2         4019.0
2           0     9814      3  ...   144.99440           2         4019.0
4           0    10589      4  ...   144.99410           2         4019.0
6           0     2142      3  ...   144.99930           2         4019.0
7           0    13335      2  ...   144.99540           2         4019.0
...       ...      ...    ...  ...         ...         ...            ...
13572     296    12200      2  ...   145.07878           2         2329.0
13573     298

#The final step! 

Finally we reach point where we can use the machine learning method (if you are a begginner, notice how much time we spent in data transformation. Yeah, welcome to my life!).

Like in many project, here, we will use the sklearn.linear_model library to retrieve the model and use training set information to calibrate the model. Then, missing values from BuildingArea are computed and assigned to the original dataframe. 






In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_training, y_train)
data.loc[data.BuildingArea.isnull(), 'BuildingArea'] = model.predict(x_test)

print(data)

print(data.BuildingArea.isnull().mean())

       Suburb  Address  Rooms  ...  Longtitude  Regionname  Propertycount
0           0    12794      2  ...   144.99840           2         4019.0
1           0     5943      2  ...   144.99340           2         4019.0
2           0     9814      3  ...   144.99440           2         4019.0
3           0     9004      3  ...   144.99690           2         4019.0
4           0    10589      4  ...   144.99410           2         4019.0
...       ...      ...    ...  ...         ...         ...            ...
13575     302     1991      4  ...   145.16761           4         7392.0
13576     305    12234      3  ...   144.87904           6         6380.0
13577     305    12745      3  ...   144.88738           6         6380.0
13578     305    13311      4  ...   144.89299           6         6380.0
13579     313    10776      4  ...   144.88449           6         6543.0

[13580 rows x 21 columns]
0.0


#DONE!

Now, it is your time to continue the example. You can use validation and/or correlation to make it more robust. Also, you can extend the idea to other numeric columns. 

Comment if you have other ideas of posts and tell me your opinion about this example!